In [10]:
 #data science - standard library
import pandas as pd
import numpy as np
import plotly.graph_objects as go


# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

from api_keys import g_key
import gmaps

import requests
import json

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
df_all = pd.read_csv("../Resources/Vertical_Merge_LatLng_AllYears.csv")
df_all.head()

,Country,Happiness Rank,Happiness Score,Economy (GDP per Capita),Social Support,Health Life Expectancy,Freedom,Trust (Government Corruption),Generosity,Year,Latitude,Longitude
0,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2015,46.818188,8.227512
1,Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2015,64.963051,-19.020835
2,Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2015,56.263920,9.501785
3,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2015,60.472024,8.468946
4,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2015,56.130366,-106.346771


In [4]:
lat = df_all.groupby('Country').Latitude.mean()
long = df_all.groupby('Country').Longitude.mean()
hap = df_all.groupby('Country')["Happiness Score"].sum()
newdf = pd.DataFrame()
newdf["Lat"]=lat
newdf["Long"]=long
newdf["Happiness Score"]=hap
newdf.reset_index()

,Country,Lat,Long,Happiness Score
0,Afghanistan,33.939110,67.709953,20.1309
1,Albania,41.153332,20.168331,28.4457
2,Algeria,28.033886,1.659626,33.3431
3,Angola,-11.202692,17.873887,15.4890
4,Argentina,-38.416097,-63.616672,38.2717
...,...,...,...,...
167,Venezuela,6.423750,-66.589730,32.7102
168,Vietnam,14.058324,108.277199,31.1265
169,Yemen,15.552727,48.516388,21.6564
170,Zambia,-13.133897,27.849332,26.6814


In [5]:
coords = newdf[["Lat","Long"]]
happy = newdf["Happiness Score"]

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coords, info_box_content=states)
# # Add the layer to the map
# fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = happy, dissipating=False, max_intensity=300)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df


,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND
...,...,...,...
217,Virgin Islands,5.08,VGB
218,West Bank,6.64,WBG
219,Yemen,45.45,YEM
220,Zambia,25.61,ZMB


In [7]:
df = df.rename(columns = {'COUNTRY': 'Country', 'CODE': 'Code'})
df

,Country,GDP (BILLIONS),Code
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND
...,...,...,...
217,Virgin Islands,5.08,VGB
218,West Bank,6.64,WBG
219,Yemen,45.45,YEM
220,Zambia,25.61,ZMB


In [8]:
results = newdf.merge(df, on='Country', how="inner")
results.head()

,Country,Lat,Long,Happiness Score,GDP (BILLIONS),Code
0,Afghanistan,33.939110,67.709953,20.1309,21.71,AFG
1,Albania,41.153332,20.168331,28.4457,13.40,ALB
2,Algeria,28.033886,1.659626,33.3431,227.80,DZA
3,Angola,-11.202692,17.873887,15.4890,131.40,AGO
4,Argentina,-38.416097,-63.616672,38.2717,536.20,ARG


In [14]:

fig = go.Figure(data=go.Choropleth(
    locations = results['Code'],
    z = results['Happiness Score'],
    text = results['Country'],
    colorscale = 'Rainbow',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'Happiness Score',
))

fig.update_layout(
    title_text='Total Happiness Score 2015-2020',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth'))
  

fig.show()